In [ ]:
import pandas as pd
import gc

In [ ]:
new_column_names = [
    'beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date',
    'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall',
    'rating', 'text', 'review'
]

output_file_path = '../../minimizedData/BA_ratings_small.csv'

def parse_ratings_file(file_path, chunk_size=15_000_012, start=0, stop=200_000_000): #chunk is a mulitple of 18 since our text 'block' size is 18
    #initialise the csv
    pd.DataFrame(columns=new_column_names).drop(['user_name','abv','date','text','review'], axis=1).to_csv(
        output_file_path, index=False
    )

    current_block = []
    data = []
    i = 0
    k = 0 
    chunk = 0
    
    with open(file_path, 'r') as file:
        for line in file:
            i += 1
            k += 1
            if i<start:
                continue
            if i % 10_000_000 == 0:
                print('Processing line: ', i)

            line = line.strip()  #remove whitespace

            if not line:  #empty line signifies end of block
                data.append(current_block)
                current_block = []
                
            if k >= chunk_size:
                k = 0
                chunk += 1
                print('Processing chunk :', chunk)
                
                df = pd.DataFrame(data, columns=new_column_names)
                df = df.drop(['user_name', 'abv', 'date', 'text', 'review'], axis=1)
                df.to_csv(output_file_path, mode='a', index=False, header=False)
                data = []  
                gc.collect()  #manually invoke garbage collection
                    
            if i+18>stop:
                break
                    
            if ': ' in line:  
                key, value = line.split(': ', 1)
                current_block.append(value)

        if not current_block: #save any leftover data at the end
            data.append(current_block)
            current_block = []
            print(k)
            k = 0
            chunk += 1 
            print('Processing chunk :', chunk)
            
            df = pd.DataFrame(data, columns=new_column_names)
            df = df.drop(['user_name', 'abv', 'date', 'text', 'review'], axis=1)
            df.to_csv(output_file_path, mode='a', index=False, header=False)
            data = []  
            gc.collect() 
            
    print('Done!')




In [2]:
parse_ratings_file('../../baseData/BeerAdvocate/ratings.txt')

Processing line:  10000000
Processing chunk : 1
Processing line:  20000000
Processing line:  30000000
Processing chunk : 2
Processing line:  40000000
Processing chunk : 3
Processing line:  50000000
Processing line:  60000000
Processing chunk : 4
Processing line:  70000000
Processing chunk : 5
Processing line:  80000000
Processing line:  90000000
Processing chunk : 6
Processing line:  100000000
Processing chunk : 7
Processing line:  110000000
Processing line:  120000000
Processing chunk : 8
Processing line:  130000000
Processing chunk : 9
Processing line:  140000000
Processing line:  150000000
Processing chunk : 10
0
Processing chunk : 11
Done!


In [ ]:
df = pd.read_csv('../../minimizedData/BA_ratings_small.csv')
df = df.drop(df.index[-1]) #get rid of a parasite row
df.to_csv('../../minimizedData/BA_ratings_small.csv') #final df is around 8 million rows
df.tail()

,beer_name,beer_id,brewery_name,brewery_id,style,user_id,appearance,aroma,palate,taste,overall,rating
8393027,Kölsch Ale,19139.0,Summit Station Restaurant & Brewery,885.0,Kölsch,jaluria.31362,3.5,3.5,3.5,4.0,4.0,3.80
8393028,Kölsch Ale,19139.0,Summit Station Restaurant & Brewery,885.0,Kölsch,dithyramb.4413,3.0,3.0,3.0,2.5,3.0,2.80
8393029,Kölsch Ale,19139.0,Summit Station Restaurant & Brewery,885.0,Kölsch,manekineko.2654,3.0,1.5,1.0,1.5,1.5,1.54
8393030,Nut Brown Ale,19140.0,Summit Station Restaurant & Brewery,885.0,English Brown Ale,redsox1539.683632,NaN,NaN,NaN,NaN,NaN,3.50
8393031,Nut Brown Ale,19140.0,Summit Station Restaurant & Brewery,885.0,English Brown Ale,dithyramb.4413,3.0,3.0,3.0,3.0,3.0,3.00


In [5]:
df.sample(3)

,beer_name,beer_id,brewery_name,brewery_id,style,user_id,appearance,aroma,palate,taste,overall,rating
6951332,The Liberator,54838.0,Short's Brewing Company,9629.0,American Double / Imperial IPA,dstc.105520,4.50,4.00,4.0,4.5,4.0,4.23
6478709,Contemplation Ale,65876.0,Brewery Vivant,24678.0,Bière de Garde,williamjbauer.655110,3.75,3.75,3.5,3.5,3.5,3.58
3922427,Ale To The Chief,43903.0,Avery Brewing Company,30.0,American Strong Ale,oxymoron.34489,4.00,4.50,4.0,4.5,4.5,4.42
